# Quantum state tomography (QST) with LS, CS and CVX

In [40]:
# To call the libraries for the GD-QST
import sys
sys.path.insert(0, '..')
# You have to change the path of the library 
import os

from qutip import * 
from itertools import *
import numpy as np
import matplotlib.pyplot as plt 
import qutip as qtp
#from qutip import basis, tensor

from qst_tec.least_square import least_square_qst
from qst_tec.compressed_sensing import compressed_sensing_qst
from qst_tec.convex_optimization_cvx import cvx_qst

import time


### First we will define the number of qubits and the measurement operators. We will use the Pauli matrices.


In [41]:
number_qubits: int = 3
dimension: int = 2**number_qubits
HS: int = dimension

# Creating the Measurement operators (Pauli matrices)

# computation basis, important for the other methods
def qubit_computation_basis(n):
    basis_states = [basis(2, i) for i in range(2)]
    return [tensor(*state) for state in product(basis_states, repeat=n)]

computation_basis = qubit_computation_basis(number_qubits)

operator_basis = [Qobj(tensor(computation_basis[i],computation_basis[j].dag()), 
                dims=([[2**number_qubits], [2**number_qubits]])) for i in range(len(computation_basis)) for j in range(len(computation_basis))]

pauli = [qeye(2), sigmax(), sigmay(), sigmaz()]
pauli_sys = [tensor(*op) for op in product(pauli, repeat=number_qubits)] # using itertools

Measu_ope = [Qobj(pauli_sys[i], dims=([[2**number_qubits], [2**number_qubits]])) for i in range(len(pauli_sys))]

print(Measu_ope[1:3])


#-----------------------------------------------------------------------
# It is necesary to transform the measurement operators from a numpy 
# type to a jax type (jax.np), that because of the jax grad 

ops_np = [op.full() for op in Measu_ope] # transforming from qutip object to numpy
ops_jnp = jnp.asarray(ops_np) # transforming from numpy to jax.np

[Quantum object: dims=[[8], [8]], shape=(8, 8), type='oper', isherm=True
Qobj data =
[[0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0.]], Quantum object: dims=[[8], [8]], shape=(8, 8), type='oper', isherm=True
Qobj data =
[[0.+0.j 0.-1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.-1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.-1.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+1.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.-1.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+1.j 0.+0.j]]]


## Implementing LS, CS, and CVX

For the case of a random density matrix for 3 qubits as the original density matrix (the one we want to reconstruct) created with QuTip $\rho_{random}$

In [42]:

max_itera: int = 300
batch_s = round(0.5*len(ops_jnp)) #len(ops_jnp)=4**N

# creating the original densirty matrix, this with a random rho from QuTip
rho_or = rand_dm(2**number_qubits)
data = qtp.expect(Measu_ope,rho_or)


#LS-QST
rho_ls, f_ls, t_ls = least_square_qst(measurement_ops=Measu_ope, basis_set=operator_basis, B=data, dimension=2**number_qubits, rho_ideal=rho_or)

#CS-QST
rho_cs, f_cs, t_cs = compressed_sensing_qst(measurement_ops=Measu_ope, basis_set=operator_basis, B=data, gamma=0, dimension=2**number_qubits, rho_ideal=rho_or)

rho_cvx, f_cvx, t_cvx = cvx_qst(measurement_ops=Measu_ope, basis_set=operator_basis, B=data, gamma=0, dimension=2**number_qubits, rho_ideal=rho_or)



In the cases of LS, CS, and CVX we just have a float value with the value of fidelity, the total time of reconstruction and the reconstructed density matrix. 

In [43]:
print('Final fidelity (LS) and average time in seconds :', f_ls, t_ls)
print('Final fidelity (CS) and average time in seconds :', f_cs, t_cs)
print('Final fidelity (CVX) and average time in seconds :', f_cvx, t_cvx)


Final fidelity (LS) and average time in seconds : 0.9999999800543798 0.09454917907714844
Final fidelity (CS) and average time in seconds : 0.9999999996685098 0.11144757270812988
Final fidelity (CVX) and average time in seconds : 0.9999999999996464 0.05239248275756836


In [44]:
print(rho_ls)

Quantum object: dims=[[8], [8]], shape=(8, 8), type='oper', isherm=True
Qobj data =
[[ 0.232854  +0.j         -0.0657598 +0.02759441j  0.0242637 +0.00755835j
  -0.0044518 +0.07482037j -0.03451106+0.02977795j  0.01486211+0.04133913j
   0.03077553-0.01350539j -0.00935112+0.06625973j]
 [-0.0657598 -0.02759441j  0.10713609+0.j         -0.0092679 -0.0151969j
   0.00838573+0.01562986j  0.02880862-0.03950405j -0.02122689-0.01395049j
  -0.00870109+0.02183851j -0.01793418+0.01776394j]
 [ 0.0242637 -0.00755835j -0.0092679 +0.0151969j   0.10567716+0.j
  -0.02327196-0.02225311j -0.01698099-0.0026105j   0.0605151 +0.02355181j
   0.00440198-0.0475387j   0.02685588-0.00101278j]
 [-0.0044518 -0.07482037j  0.00838573-0.01562986j -0.02327196+0.02225311j
   0.0988422 +0.j         -0.00556328-0.00828518j  0.04084065+0.02315615j
   0.00937139-0.00296617j  0.03044136-0.00049055j]
 [-0.03451106-0.02977795j  0.02880862+0.03950405j -0.01698099+0.0026105j
  -0.00556328+0.00828518j  0.0856197 +0.j         -0.025